In [622]:
# Parameters
GROUP_ID = '43'
ALGORITHM = 'bt' # bt | fc | ac3 | sa | ga
PUZZLE_TYPE = 'extreme' # easy | medium | hard | extreme
PUZZLE_PATH = 'C:\\Users\\Asus\\Downloads\\AiSudokuSolver-main\\AiSudokuSolver-main\\puzzles\\extreme\\Evil-P4.txt'

In [623]:
from io_1 import read_puzzle, write_puzzle, build_output_filename
import numpy as np
import hashlib
import random
from random import choice
# Deterministic seed from parameters
_seed = int(hashlib.md5(f"{GROUP_ID}:{ALGORITHM}:{PUZZLE_PATH}".encode()).hexdigest()[:8], 16)

In [624]:
def flag_Initial(grid):
    flag = np.zeros((9,9), dtype=int)
    for i in range (0,9):
        for j in range (0,9):
            if grid[i][j] != 0:
                flag[i,j] = 1
    return flag

In [625]:
def fitness(grid):
    ErrorCount = 0
    total = 0
    for i in range(0,9):
        ErrorCount = (9 - len(np.unique(grid[:,i]))) + (9 - len(np.unique(grid[i,:])))
        total = total + ErrorCount
    return total

In [626]:
def getBlocks(grid):
    answer = []
    for r in range(3):
        for c in range(3):
            block = []
            for i in range(3):
                for j in range(3):
                    block.append(grid[3*r + i][3*c + j])
            answer.append(block)
    return answer

In [627]:
def fillrandom(blocks):
    for block in blocks:
        for j in range(0,9):
            if block[j] == 0:
                    block[j] = choice([i for i in range(1,10) if i not in block])
    randomgrid = []
    for block_row in range(3):       
        for row_in_block in range(3): 
            row = []
            for block_col in range(3):  
                block_index = block_row * 3 + block_col
                row.extend(blocks[block_index][row_in_block*3 : (row_in_block+1)*3])
            randomgrid.append(row)
    randomgrid = np.array(randomgrid)
    return randomgrid

In [628]:
def flip(gridtemp , flag):
    temporary = gridtemp.copy()
    while True:
        row1 =  random.randint(0,8)
        col1 =  random.randint(0,8)
        while True:
            r = (row1 // 3)*3 + random.randint(0,2)
            c = (col1 // 3)*3 + random.randint(0,2)
            if (r,c) != (row1,col1):
                row2 = r
                col2 = c
                break
        if flag[row1][col1] != 1 and flag[row2][col2] != 1:
            # print(f"Swapping ({row1},{col1}) with ({row2},{col2})")
            temp = temporary[row1][col1]
            temporary[row1][col1] = temporary[row2][col2]
            temporary[row2][col2] = temp
            return temporary
        
    



In [629]:
initialgrid = read_puzzle(PUZZLE_PATH)
print(initialgrid)
grid = initialgrid.copy()
blocks = getBlocks(grid)
print(blocks)
flag = flag_Initial(grid)
# randomgrid = []
# for block_row in range(3):       
#     for row_in_block in range(3): 
#         row = []
#         for block_col in range(3):  
#             block_index = block_row * 3 + block_col
#             row.extend(blocks[block_index][row_in_block*3 : (row_in_block+1)*3])
#         randomgrid.append(row)
# randomgrid = np.array(randomgrid)



[[0, 0, 3, 9, 1, 4, 6, 0, 0], [6, 0, 0, 0, 0, 0, 5, 0, 4], [0, 1, 0, 0, 0, 6, 0, 9, 3], [0, 4, 2, 0, 0, 7, 0, 0, 5], [0, 3, 8, 0, 0, 0, 9, 7, 0], [9, 0, 0, 5, 0, 0, 4, 2, 0], [8, 7, 0, 2, 0, 0, 0, 4, 0], [3, 0, 6, 0, 0, 0, 0, 0, 8], [0, 0, 5, 1, 8, 3, 7, 0, 0]]
[[0, 0, 3, 6, 0, 0, 0, 1, 0], [9, 1, 4, 0, 0, 0, 0, 0, 6], [6, 0, 0, 5, 0, 4, 0, 9, 3], [0, 4, 2, 0, 3, 8, 9, 0, 0], [0, 0, 7, 0, 0, 0, 5, 0, 0], [0, 0, 5, 9, 7, 0, 4, 2, 0], [8, 7, 0, 3, 0, 6, 0, 0, 5], [2, 0, 0, 0, 0, 0, 1, 8, 3], [0, 4, 0, 0, 0, 8, 7, 0, 0]]


In [630]:
# current_grid = fillrandom(blocks)
# print(current_grid)
# newgrid = flip(current_grid , flag)
# print(newgrid)

In [631]:

current_grid = fillrandom(blocks)
temperature = 1
cooling_rate = 0.95
iteration = 200
stuck_count = 0
loop_count = 0
while True:
    for i in range (iteration):
        stuck_count += 1
        loop_count += 1
        newgrid = flip(current_grid, flag)
        if fitness(newgrid) < fitness(current_grid):
            current_grid = newgrid
            print("Accepted better solution")
        else:
            acceptance_probability = np.exp((fitness(current_grid) - fitness(newgrid)) / temperature)
            if random.random() < acceptance_probability:
                current_grid = newgrid
                print("Accepted worse solution")

        if fitness(current_grid) == 0:
            print("Solution found")
            print(f"Total loops: {loop_count}")
            print(current_grid)
            break
        print(fitness(current_grid),stuck_count,loop_count)
    temperature = temperature * cooling_rate 
    print(f"End of iteration, temperature: {temperature}")
    if stuck_count == 30000:
        current_grid = fillrandom(blocks)
        stuck_count = 0
        print("Restarting with new random grid")
        temperature = temperature + 2
    
    if fitness(current_grid) == 0:
        break

40 1 1
Accepted better solution
39 2 2
Accepted worse solution
39 3 3
Accepted better solution
37 4 4
37 5 5
37 6 6
Accepted better solution
35 7 7
Accepted better solution
34 8 8
34 9 9
34 10 10
34 11 11
34 12 12
34 13 13
Accepted better solution
32 14 14
32 15 15
32 16 16
Accepted worse solution
33 17 17
Accepted worse solution
33 18 18
Accepted worse solution
34 19 19
Accepted worse solution
35 20 20
Accepted worse solution
35 21 21
35 22 22
Accepted better solution
34 23 23
34 24 24
Accepted worse solution
34 25 25
Accepted worse solution
34 26 26
34 27 27
34 28 28
Accepted better solution
33 29 29
33 30 30
33 31 31
33 32 32
Accepted worse solution
33 33 33
Accepted worse solution
33 34 34
Accepted better solution
32 35 35
Accepted worse solution
33 36 36
33 37 37
33 38 38
33 39 39
Accepted worse solution
35 40 40
Accepted better solution
32 41 41
Accepted better solution
31 42 42
31 43 43
Accepted worse solution
31 44 44
31 45 45
31 46 46
31 47 47
Accepted better solution
28 48 48